In [11]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv

In [12]:
load_dotenv()

True

In [7]:
import os


In [4]:
# Extract drata from pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf("../data/")

In [6]:
# Create chunks
def create_text_chunks(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = create_text_chunks(extracted_data)
print(len(text_chunks))

1423


In [8]:
# Vector Embeddings ( Download Embedding model)
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [9]:
embeddings = download_hugging_face_embeddings()

/Users/dhanushlalitha/miniconda3/envs/chatbot/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/dhanushlalitha/miniconda3/envs/chatbot/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
pc = Pinecone()

In [10]:
index_name = 'chatbot-index'
# index = pc.Index(index_name)
# index.delete(delete_all=True)

{}

In [14]:

# Create embeddings for each text chunk and store
doc_search = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [15]:
doc_search = PineconeVectorStore.from_existing_index(index_name, embedding=embeddings)


In [17]:
query = "What is Type1 Diabetes"
docs = doc_search.similarity_search(query=query, k=2)
print(docs)

[Document(page_content='20.  Park Y, Wintergerst kA, Zhou Z. Clinical heterogeneity of type 1 diabetes (T1D) found in Asia. Diabetes \nMetab Res Rev 2017;33.\n21. Imagawa A, Hanafusa T, Miyagawa J, Matsuzawa Y. A novel subtype of type 1 diabetes mellitus characterized by a rapid onset and an absence of diabetes-related antibodies. Osaka IDDM Study Group. N Engl J Med. 2000;342:301-7.'), Document(page_content='the epidemiology and diagnosis and differential diagnosis of type 1 diabetes to lifestyle, diet and exercise, insulin, monitoring, acute complications, microvascular complications including retinopathy, diabetic kidney disease and nerve diseases, macro vascular complications, education, problem of T1D in special groups like pregnancy, travel and driving. Thus, it covers practically the whole gamut of type 1 diabetes. It also incorporates most of the published Indian references on the subject.')]


In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [21]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.7})

In [22]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=doc_search.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [23]:

user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

/Users/dhanushlalitha/miniconda3/envs/chatbot/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  Type 1 diabetes symptoms can include increased thirst and urination, fatigue, weight loss, blurred vision, and slow healing of cuts and wounds.
